<a href="https://colab.research.google.com/github/omkart0124/ML_project/blob/main/Customer_CHURN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
df = pd.read_csv("churn.csv")
df.head(2)

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,Yes,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,No,Yes,No,No,No,One year,No,Mailed check,56.95,1889.50,No


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7043 entries, 0 to 7042
Data columns (total 21 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   customerID        7043 non-null   object 
 1   gender            7043 non-null   object 
 2   SeniorCitizen     7043 non-null   int64  
 3   Partner           7043 non-null   object 
 4   Dependents        7043 non-null   object 
 5   tenure            7043 non-null   int64  
 6   PhoneService      7043 non-null   object 
 7   MultipleLines     7043 non-null   object 
 8   InternetService   7043 non-null   object 
 9   OnlineSecurity    7043 non-null   object 
 10  OnlineBackup      7043 non-null   object 
 11  DeviceProtection  7043 non-null   object 
 12  TechSupport       7043 non-null   object 
 13  StreamingTV       7043 non-null   object 
 14  StreamingMovies   7043 non-null   object 
 15  Contract          7043 non-null   object 
 16  PaperlessBilling  7043 non-null   object 


In [ ]:
df.loc[df.Churn=='No','Churn'] = 0 
df.loc[df.Churn=='Yes','Churn'] = 1

In [ ]:
# Convert 'No internet service' to 'No' for the below mentioned columns
cols = ['OnlineBackup', 'StreamingMovies','DeviceProtection',
                'TechSupport','OnlineSecurity','StreamingTV']
for i in cols : 
    df[i]  = df[i].replace({'No internet service' : 'No'})

In [ ]:
# Replace all the spaces with null values
df['TotalCharges'] = df["TotalCharges"].replace(" ",np.nan)

In [ ]:
# Drop null values of 'Total Charges' feature
df = df[df["TotalCharges"].notnull()]
df = df.reset_index()[df.columns]

OneHotEncoding

In [ ]:
df = pd.get_dummies(df, columns = ['Contract','Dependents','DeviceProtection','gender',
                                                        'InternetService','MultipleLines','OnlineBackup',
                                                        'OnlineSecurity','PaperlessBilling','Partner',
                                                        'PaymentMethod','PhoneService','SeniorCitizen',
                                                        'StreamingMovies','StreamingTV','TechSupport'],
                              drop_first=True)

In [ ]:
#Perform Feature Scaling and One Hot Encoding
from sklearn.preprocessing import StandardScaler

In [ ]:
standardScaler = StandardScaler()
columns_for_ft_scaling = ['tenure', 'MonthlyCharges', 'TotalCharges']

In [ ]:
#Apply the feature scaling operation on dataset using fit_transform() method
df[columns_for_ft_scaling] = standardScaler.fit_transform(df[columns_for_ft_scaling])

In [ ]:
df.drop(["customerID"],axis=1, inplace=True)#-------Dropped customerID

In [ ]:
df.head()

,tenure,MonthlyCharges,TotalCharges,Churn,Contract_One year,Contract_Two year,Dependents_Yes,DeviceProtection_Yes,gender_Male,InternetService_Fiber optic,InternetService_No,MultipleLines_No phone service,MultipleLines_Yes,OnlineBackup_Yes,OnlineSecurity_Yes,PaperlessBilling_Yes,Partner_Yes,PaymentMethod_Credit card (automatic),PaymentMethod_Electronic check,PaymentMethod_Mailed check,PhoneService_Yes,SeniorCitizen_1,StreamingMovies_Yes,StreamingTV_Yes,TechSupport_Yes
0,-1.280248,-1.161694,-0.994194,0,0,0,0,0,0,0,0,1,0,1,0,1,1,0,1,0,0,0,0,0,0
1,0.064303,-0.260878,-0.173740,0,1,0,0,1,1,0,0,0,0,0,1,0,0,0,0,1,1,0,0,0,0
2,-1.239504,-0.363923,-0.959649,1,0,0,0,0,1,0,0,0,0,1,1,1,0,0,0,1,1,0,0,0,0
3,0.512486,-0.747850,-0.195248,0,1,0,0,1,1,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,1
4,-1.239504,0.196178,-0.940457,1,0,0,0,0,0,1,0,0,0,0,0,1,0,0,1,0,1,0,0,0,0


Looking for Outliers

In [ ]:
num = ["tenure", "MonthlyCharges", "TotalCharges"]
dff_num = df[num]
dff_num.describe()

Q1 = dff_num.quantile(0.25)
Q3 = dff_num.quantile(0.75)
IQR = Q3 - Q1
IQR
((dff_num < (Q1 - 1.5*IQR)) | (dff_num > (Q3 + 1.5 * IQR))).any()

tenure            False
MonthlyCharges    False
TotalCharges      False
dtype: bool

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics

In [ ]:
X = df.drop('Churn', axis=1)
y = df["Churn"]
y = y.astype('int')

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=50)

###LogisticRegression

In [ ]:
log = LogisticRegression()
log.fit(X_train,y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [ ]:
log.predict_proba(x_test)

array([[0.63510713, 0.36489287],
       [0.98232154, 0.01767846],
       [0.6287899 , 0.3712101 ],
       ...,
       [0.66173507, 0.33826493],
       [0.77520079, 0.22479921],
       [0.7564395 , 0.2435605 ]])

In [ ]:
log.predict(x_test) 

array([0, 0, 0, ..., 0, 0, 0])

In [ ]:
y_hat = log.predict(X_test)  # storing values into y_hat 

In [ ]:
from sklearn.metrics import (accuracy_score, precision_score, recall_score, f1_score)

In [ ]:
# Find Accuracy using accuracy_score method
logmodel_accuracy = (metrics.accuracy_score(y_test, y_hat))

###SVM

In [ ]:
svcmodel = SVC(kernel='linear', random_state=50, probability=True)
svcmodel.fit(X_train,y_train)

SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='linear',
    max_iter=-1, probability=True, random_state=50, shrinking=True, tol=0.001,
    verbose=False)

In [ ]:
#Predict the value for new, unseen data
svc_pred = svcmodel.predict(X_test)

In [ ]:
# Find Accuracy using accuracy_score method
svc_accuracy = (accuracy_score(y_test, svc_pred))

In [ ]:
svc_accuracy

0.8066350710900474

###K-Nearest Neighbor Model

In [ ]:
#Fit the K-Nearest Neighbor Model
from sklearn.neighbors import KNeighborsClassifier
knnmodel = KNeighborsClassifier(n_neighbors=5, metric='minkowski', p=2) #p=2 represents Euclidean distance, p=1 represents Manhattan Distance
knnmodel.fit(X_train, y_train) 

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=5, p=2,
                     weights='uniform')

In [ ]:
knn_pred = knnmodel.predict(X_test)

In [ ]:
knn_accuracy = (accuracy_score(y_test, knn_pred))

In [ ]:
knn_accuracy

0.7687203791469195

###Decision Tree

In [ ]:
#Fit the Decision Tree Classification Model
from sklearn.tree import DecisionTreeClassifier
dtmodel = DecisionTreeClassifier(criterion = "gini", random_state = 50)
dtmodel.fit(X_train, y_train)

DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                       max_depth=None, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=50, splitter='best')

In [ ]:
dt_pred = dtmodel.predict(X_test)

In [ ]:
dt_accuracy = (accuracy_score(y_test, dt_pred))
dt_accuracy

0.7327014218009479

###Random Forest

In [ ]:
#Fit the Random Forest Classification Model
from sklearn.ensemble import RandomForestClassifier
rfmodel = RandomForestClassifier(n_estimators = 100, criterion = 'entropy', random_state = 0)
rfmodel.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='entropy', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=0, verbose=0,
                       warm_start=False)

In [ ]:
rf_pred = rfmodel.predict(X_test)

In [ ]:
rf_accuracy = (accuracy_score(y_test, rf_pred))
rf_accuracy

0.7938388625592417

###Compare Several models according to their Accuracies

In [ ]:
# Compare Several models according to their Accuracies
Model_Comparison = pd.DataFrame({
    'Model': ['Logistic Regression', 'Support Vector Machine', 'K-Nearest Neighbor', 
              'Decision Tree', 'Random Forest'],
    'Score': [logmodel_accuracy, svc_accuracy, knn_accuracy, 
              dt_accuracy, rf_accuracy]})
Model_Comparison_df = Model_Comparison.sort_values(by='Score', ascending=False)
Model_Comparison_df = Model_Comparison_df.set_index('Score')
Model_Comparison_df.reset_index()

,Score,Model
0,0.811374,Logistic Regression
1,0.806635,Support Vector Machine
2,0.793839,Random Forest
3,0.768720,K-Nearest Neighbor
4,0.732701,Decision Tree


##Conclusion :


I have used Logistic Regression, SVM, Random Forest, KNN and Decision tree and among all of them I Found Logistic Regression get best accuracy.